In [3]:
import logging
import warnings

In [4]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
warnings.filterwarnings('ignore')

In [5]:
logger.info("start")

INFO:__main__:start


In [6]:
from datasets import load_dataset

INFO:datasets:PyTorch version 2.7.1 available.


In [7]:
dataset = load_dataset('jfleg')

In [8]:
dataset["validation"]["sentence"][0]

'So I think we can not live if old people could not find siences and tecnologies and they did not developped . '

In [9]:
dataset["validation"]["corrections"][0]

['So I think we would not be alive if our ancestors did not develop sciences and technologies . ',
 'So I think we could not live if older people did not develop science and technologies . ',
 'So I think we can not live if old people could not find science and technologies and they did not develop . ',
 'So I think we can not live if old people can not find the science and technology that has not been developed . ']

In [10]:
len(dataset["validation"])

755

In [11]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['sentence', 'corrections'],
        num_rows: 755
    })
    test: Dataset({
        features: ['sentence', 'corrections'],
        num_rows: 748
    })
})

In [12]:
from transformers import AutoTokenizer, T5ForConditionalGeneration
# MODELNAME = "vennify/t5-base-grammar-correction"
MODELNAME = "t5-small"
PREFIX = "grammar: "
tokenizer = AutoTokenizer.from_pretrained(MODELNAME)
model = T5ForConditionalGeneration.from_pretrained(MODELNAME)

In [13]:
# Define the input text with a task prefix
input_text = PREFIX + "he go to school yesterday."

# Tokenize the input
input_ids = tokenizer.encode(
    input_text, return_tensors="pt", max_length=128, truncation=True)

# Generate output (corrected text)
output = model.generate(
    input_ids,
    max_length=128,
    num_beams=5,  # Beam search for better quality
    early_stopping=True,
    repetition_penalty=2.5  # Penalize repetitive output
)

# Decode the generated text
corrected_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(f"Original: he go to school yesterday.")
print(f"Corrected: {corrected_text}")

Original: he go to school yesterday.
Corrected: grammar: he go to school yesterday.


In [14]:
tokenizer.decode(output[0], skip_special_tokens=True)

'grammar: he go to school yesterday.'

In [15]:
ds = load_dataset("dim/grammarly_coedit")

In [16]:
input_ids

tensor([[19519,    10,     3,    88,   281,    12,   496,  4981,     5,     1]])

In [17]:
type(dataset["validation"]["sentence"])

list

In [18]:
from transformers import T5Config

In [19]:
print(T5Config())

T5Config {
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "transformers_version": "4.52.4",
  "use_cache": true,
  "vocab_size": 32128
}



In [20]:
from torch.utils.data import DataLoader, Dataset
import torch

In [27]:
class GrammarCorrectionDataset(Dataset):
    def __init__(self, tokenizer, input_text, target_text, max_length=256):
        self.tokenizer = tokenizer
        self.input_text = input_text
        self.target_text = target_text
        self.max_length = max_length

    def __len__(self):
        return len(self.input_text)

    def __getitem__(self, id):
        source = str("grammar: " + self.input_text[id])
        target = str(self.target_text[id])

        source_tokens = self.tokenizer(
            source, return_tensors="pt", max_length=self.max_length,
            padding="max_length", truncation=True, return_attention_mask=True)

        target_tokens = self.tokenizer(
            target, return_tensors="pt", max_length=self.max_length,
            padding="max_length", truncation=True, return_attention_mask=True)

        labels = target_tokens["input_ids"].clone()
        labels[labels == tokenizer.pad_token_id] = -100

        return {
            "input_ids": source_tokens["input_ids"].flatten(),
            "attention_mask": source_tokens["attention_mask"].flatten(),
            "labels": labels.flatten(),
        }

In [28]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['sentence', 'corrections'],
        num_rows: 755
    })
    test: Dataset({
        features: ['sentence', 'corrections'],
        num_rows: 748
    })
})

In [29]:
train_source = dataset["validation"]["sentence"]
train_labels = dataset["validation"]["corrections"]

val_source = dataset["test"]["sentence"]
val_labels = dataset["test"]["corrections"]

In [36]:
train_dataset = GrammarCorrectionDataset(tokenizer, train_source, train_labels)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

In [37]:

train_loader

In [40]:
for i, data in enumerate(train_loader):
    print(data)
    break

{'input_ids': tensor([[19519,    10,    94,    47,     3,     9,  4360,   594,     3,     5,
             1,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [33]:
dataset["validation"]["sentence"][0]  # type: ignore

'So I think we can not live if old people could not find siences and tecnologies and they did not developped . '

In [ ]:
tokenizer.encode(
    # type: ignore
    str("grammar: " + dataset["validation"]["sentence"][0]),  max_length=128, padding="max_length", truncation=True, return_tensors="pt")

tensor([[19519,    10,   264,    27,   317,    62,    54,    59,   619,     3,
            99,   625,   151,   228,    59,   253,   108,  1433,     7,    11,
             3,  5822,    29,  4137,     7,    11,    79,   410,    59,  1344,
          3138,     3,     5,     1,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  